In [1]:
import pandas as pd

municipios = pd.read_excel('resources/elecciones.xlsx', index_col=False)

In [60]:
# Arreglor el Nombre de las columnas de la tabla para poder acceder a ellas usando el "."
col_names = ['Nombre de Comunidad', 'Código de Provincia', 'Nombre de Provincia',
'Código de Municipio', 'Nombre de Municipio', 'Población',
'Número de mesas', 'Total censo electoral', 'Total votantes',
'Votos válidos', 'Votos a candidaturas', 'Votos en blanco',
'Votos nulos',
'PSOE',
'PP',
'Cs',
'PODEMOS-IU-EQUO',
'VOX',
'ERC-SOBIRANISTES',
'ECP-GUANYEM EL CANVI',
'JxCAT-JUNTS',
'EAJ-PNV',
'EH Bildu',
'PODEMOS-EU-MAREAS EN COMÚN-EQUO',
'CCa-PNC',
'NA+',
'COMPROMÍS 2019',
'PRC',
'PACMA',
'FRONT REPUBLICÀ',
'BNG',
'RECORTES CERO-GV',
'NCa',
'PACT',
'ARA-MES-ESQUERRA',
'GBAI',
'PUM+J',
'EN MAREA',
'PCTE',
'EL PI',
'AxSI',
'PCOE',
'PCPE',
'AVANT ADELANTE LOS VERDES',
'EB',
'CpM',
'SOMOS REGIÓN',
'PCPA',
'PH',
'UIG-SOM-CUIDES',
'ERPV',
'IZQP',
'PCPC',
'AHORA CANARIAS',
'CxG',
'PPSO',
'CNV',
'PREPAL',
'C.Ex-C.R.Ex-P.R.Ex',
'PR+',
'P-LIB',
'CILU-LINARES',
'ANDECHA ASTUR',
'JF',
'PYLN',
'FIA',
'FE de las JONS',
'F8',
'SOLIDARIA',
'DPL',
'UNIÓN REGIONALISTA',
'centrados',
'VOU',
'DP',
'PDSJE-UDEC',
'IZAR',
'RISA',
'C 21',
'+MAS+',
'UDT']
fix_col_names = list(map(lambda nombre: nombre.replace(' ','_'), col_names))
municipios.columns = fix_col_names

# Arreglar el los espacios adicionales en los valores string de la tabla (columnas 0, 2 y 4)
municipios.Nombre_de_Comunidad = municipios.Nombre_de_Comunidad.apply(lambda i: i.strip())
municipios.Nombre_de_Provincia = municipios.Nombre_de_Provincia.apply(lambda i: i.strip())
municipios.Nombre_de_Municipio = municipios.Nombre_de_Municipio.apply(lambda i: i.strip())

# La columna Codigo_de_Provincia debe ser string, no se deberia poder operaciones matematicas sobre ella
municipios.Código_de_Provincia = [str(i) for i in municipios.Código_de_Provincia]

In [61]:
municipios.Código_de_Provincia[0]
# municipios[municipios.Nombre_de_Provincia == 'Madrid']

'4'

In [49]:
municipios.head()

,Nombre_de_Comunidad,Código_de_Provincia,Nombre_de_Provincia,Código_de_Municipio,Nombre_de_Municipio,Población,Número_de_mesas,Total_censo_electoral,Total_votantes,Votos_válidos,...,UNIÓN_REGIONALISTA,centrados,VOU,DP,PDSJE-UDEC,IZAR,RISA,C_21,+MAS+,UDT
0,Andalucía,4,Almería,1,Abla ...,1235,2,1014,768,763,...,0,0,0,0,0,0,0,0,0,0
1,Andalucía,4,Almería,2,Abrucena ...,1150,2,1039,798,792,...,0,0,0,0,0,0,0,0,0,0
2,Andalucía,4,Almería,3,Adra ...,24859,29,17540,12174,12051,...,0,0,0,0,0,0,0,0,0,0
3,Andalucía,4,Almería,4,Albánchez ...,753,1,400,320,315,...,0,0,0,0,0,0,0,0,0,0
4,Andalucía,4,Almería,5,Alboloduy ...,621,1,533,448,442,...,0,0,0,0,0,0,0,0,0,0
